# 코로나 일별 확진자 증감 추세 외부 데이터 

In [ ]:
import requests
import json
import pandas as pd
import requests, bs4
import pandas as pd
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import numpy as np

from urllib.request import Request, urlopen 
from urllib.parse import urlencode, quote_plus



In [ ]:

import requests
#수정
xmlUrl='http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19SidoInfStateJson'

#수정
service_key=unquote('b8sLGVKSMcrfCRS5JTtLN8iiuZrgcpTjGbMIU8UUnlE85Z5hif86%2FWb8tFU698WvQWmE01EMuSlXbo%2B92wxVpg%3D%3D'
)



In [ ]:
#수정
queryParams = '?' + urlencode({quote_plus('ServiceKey') : service_key, 
                               quote_plus('pageNo') : '1',
                               quote_plus('numOfRows') : '10', 
                               quote_plus('startCreateDt') : '20210301',
                               quote_plus('endCreateDt') : '20210630',
                               quote_plus('DEF_CNT'):''
                              })


In [ ]:
response = requests.get(xmlUrl + queryParams).text.encode('utf-8')
xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')
print(xmlobj)

<?xml version="1.0" encoding="utf-8"?>
<response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><createDt>2021-06-30 09:00:00.000</createDt><deathCnt>2018</deathCnt><defCnt>156960</defCnt><gubun>합계</gubun><gubunCn>合计</gubunCn><gubunEn>Total</gubunEn><incDec>794</incDec><isolClearCnt>147683</isolClearCnt><isolIngCnt>7259</isolIngCnt><localOccCnt>759</localOccCnt><overFlowCnt>35</overFlowCnt><qurRate>302.73</qurRate><seq>10032</seq><stdDay>2021년 06월 30일 00시</stdDay><updateDt>2021-09-16 10:11:33.897</updateDt></item><item><createDt>2021-06-30 09:00:00.000</createDt><deathCnt>513</deathCnt><defCnt>49986</defCnt><gubun>서울</gubun><gubunCn>首尔</gubunCn><gubunEn>Seoul</gubunEn><incDec>375</incDec><isolClearCnt>46496</isolClearCnt><isolIngCnt>2977</isolIngCnt><localOccCnt>368</localOccCnt><overFlowCnt>7</overFlowCnt><qurRate>513.55</qurRate><seq>10031</seq><stdDay>2021년 06월 30일 00시</stdDay><updateDt>2021-09-16 10:11:33.897</updateDt></item><i

In [ ]:

pd.DataFrame(xmlobj)

C:\Users\ns451\Documents\anaconda_1\lib\site-packages\pandas\core\internals\construction.py:309: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  values = np.array([convert(v) for v in values])


,0,1
0,"[[00], [NORMAL SERVICE.]]",[[[<createDt>2021-06-30 09:00:00.000</createDt...


In [ ]:
rows = xmlobj.findAll('item')

In [ ]:
columns=rows[0].find_all()
columns

[<createDt>2021-06-30 09:00:00.000</createDt>,
 <deathCnt>2018</deathCnt>,
 <defCnt>156960</defCnt>,
 <gubun>합계</gubun>,
 <gubunCn>合计</gubunCn>,
 <gubunEn>Total</gubunEn>,
 <incDec>794</incDec>,
 <isolClearCnt>147683</isolClearCnt>,
 <isolIngCnt>7259</isolIngCnt>,
 <localOccCnt>759</localOccCnt>,
 <overFlowCnt>35</overFlowCnt>,
 <qurRate>302.73</qurRate>,
 <seq>10032</seq>,
 <stdDay>2021년 06월 30일 00시</stdDay>,
 <updateDt>2021-09-16 10:11:33.897</updateDt>]

In [ ]:
rowList = []
nameList = []
columnList = []

rowsLen = len(rows)
for i in range(0, rowsLen):
    columns = rows[i].find_all()
    
    columnsLen = len(columns)
    for j in range(0, columnsLen):
        # 첫 번째 행 데이터 값 수집 시에만 컬럼 값을 저장한다. (어차피 rows[0], rows[1], ... 모두 컬럼헤더는 동일한 값을 가지기 때문에 매번 반복할 필요가 없다.)
        if i == 0:
            nameList.append(columns[j].name)
        # 컬럼값은 모든 행의 값을 저장해야한다.    
        eachColumn = columns[j].text
        columnList.append(eachColumn)
    rowList.append(columnList)
    columnList = []    # 다음 row의 값을 넣기 위해 비워준다. (매우 중요!!)
    


In [ ]:
result= pd.DataFrame(rowList, columns=nameList)
result

#확진자 수: DEF_CNT


,createDt,deathCnt,defCnt,gubun,gubunCn,gubunEn,incDec,isolClearCnt,isolIngCnt,localOccCnt,overFlowCnt,qurRate,seq,stdDay,updateDt
0,2021-06-30 09:00:00.000,2018,156960,합계,合计,Total,794,147683,7259,759,35,302.73,10032,2021년 06월 30일 00시,2021-09-16 10:11:33.897
1,2021-06-30 09:00:00.000,513,49986,서울,首尔,Seoul,375,46496,2977,368,7,513.55,10031,2021년 06월 30일 00시,2021-09-16 10:11:33.897
2,2021-06-30 09:00:00.000,124,6221,부산,釜山,Busan,14,5852,245,13,1,182.34,10030,2021년 06월 30일 00시,2021-09-16 10:11:33.897
3,2021-06-30 09:00:00.000,222,10598,대구,大邱,Daegu,11,10294,82,11,0,434.97,10029,2021년 06월 30일 00시,2021-09-16 10:11:33.897
4,2021-06-30 09:00:00.000,61,6767,인천,仁川,Incheon,30,6461,245,29,1,228.92,10028,2021년 06월 30일 00시,2021-09-16 10:11:33.897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2313,2021-03-01 09:00:00.000,8,861,전남,全南,Jeollanam-do,2,771,82,2,0,46.17,7719,2021년 03월 01일 00시,2021-09-16 10:11:33.897
2314,2021-03-01 09:00:00.000,71,3266,경북,庆北,Gyeongsangbuk-do,13,3050,145,10,3,122.67,7718,2021년 03월 01일 00시,2021-09-16 10:11:33.897
2315,2021-03-01 09:00:00.000,11,2188,경남,庆南,Gyeongsangnam-do,5,2110,67,4,1,65.09,7717,2021년 03월 01일 00시,2021-09-16 10:11:33.897
2316,2021-03-01 09:00:00.000,1,570,제주,济州,Jeju,0,543,26,0,0,84.98,7716,2021년 03월 01일 00시,2021-09-16 10:11:33.897


In [ ]:
result['createDt']=result['createDt'].str[0:10]


In [ ]:
result1=result[['createDt','incDec','gubun']]

In [ ]:
#gubun=합계 행 제거
result1=result1[result1['gubun']!='합계']

In [ ]:
#필요 행만 추출
result1=result1[(result1['gubun']=='서울')|(result1['gubun']=='경기')|(result1['gubun']=='서울')|(result1['gubun']=='경남')|(result1['gubun']=='충북')]

In [ ]:
#저장
result1=pd.DataFrame(result1)
result1.to_csv('C:/Users/ns451/Documents/프로젝트/대한통운 프로젝트[공모전]/코로나_확진자_데이터.csv',index=False, header=True)
